# CZI Software Mentions Dataset - Interacting with the Dataset


This notebook offers examples of **interacting** with the <b>CZI Software Mentions dataset </b><br>
The <b>CZI Software Mentions dataset </b> is a large dataset of software mentions mined from the literature. 

**Dataset Overview**: Plain-text software mentions are extracted with a trained [SciBERT](#references_scibert)model from several sources: the NIH PubMed Central collection and from papers provided by various publishers to the Chan Zuckerberg Initiative. The dataset provides sources, context and metadata, and, for a number of mentions, the disambiguated software entities and links. Full description of the dataset, methodology, algorihms and evaluation used to create the dataset can be found in our preprint, [A large dataset of software mentions in the biomedical literature](Link) and on our [Github page](https://github.com/chanzuckerberg/software-mentions). 


**The notebook is structured and offers the following information and examples, as follows:**

1. [Interacting with dataset](#dataset_interaction)
    1. [Description of the dataset](#dataset_interaction_description)
    2. [raw files](#dataset_interaction_raw)
        1. Example: [Query the dataset for a particular plain-text software mention](#dataset_interaction_raw_query_scipy)
        2. Example: [Retrieve texts in which a particular plain-text software mention appears](#dataset_interaction_raw_samples_scipy)
        3. Example: [Retrieve the most frequent plain-text software mentions in the corpus](#dataset_interaction_raw_most_frequent_mentions)
        4. Motivation: [Understanding curation labels](#dataset_interaction_raw_curation_labels)
    3. [disambiguated files](#dataset_interaction_disambiguated)
        1. Motivation: [Why do we need disambiguation? Examples showcasing string variation](#dataset_interaction_disambiguated_motivation)
        2. Example: [Query the dataset for a particular software, including all string variations](#dataset_interaction_disambiguated_dataset_query)
        3. Example: [Examples of disambiguated software terms](#dataset_interaction_disambiguated_examples)
        4. Example: [Retrieve the most frequent software mentions based on disambiguated dataset](#dataset_interaction_disambiguated_most_frequent_mentions)
    4. [linked files](#dataset_interaction_linked)
        1. Example: [Query the linked data](#dataset_interaction_linked_query)
        2. Example: [Exploration of metadata fields](#dataset_interaction_linked_metadata_fields)
2. Example: [Query the dataset for a particular software, including all string variations, and links](#dataset_interaction_linked_query) (Example for **scikit-learn**)

There is a different notebook, [CZI Software Mentions Dataset - Sample Use Cases](#link_here), that offers sample use cases for the dataset.

**The full list of resources we have available for the dataset is**:
1. [Preprint: A large dataset of software mentions in the biomedical literature](link)
2. [Github Repository](https://github.com/chanzuckerberg/software-mentions)
3. [Dataset README.md](link)
4. [CZI Software Mentions Dataset - Interacting with the Dataset](#link_here) - Jupyter Notebook
5. [CZI Software Mentions Dataset - Sample Use Cases](#link_here) - Jupyter Notebook

For questions, please contact aistrate@chanzuckerberg.com

In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 1000)

<a id='dataset_interaction'></a>

## Interacting with the dataset
We offer a brief overview of the dataset below. For a full description, including detailed information about the available files and fields, and how they were obtained, please consult the dataset [README.md](#Linkhere) file, or the Appendix section of our [preprint](link)

<a id='dataset_interaction_description'></a>
### Dataset Description


The notebook assumes that the dataset files are stored in a folder `data` that sits as the same level as the `sample_notebooks` directory. The assumed directory structure is the following:

- `sample_notebooks`
-  `data` 
    - `raw`
        - comm_raw.tsv.gz
        - non_comm_raw.tsv.gz
        - publishers_collection_raw.tsv.gz
    - `disambiguated`
        - comm_disambiguated.tsv.gz
    - `linked`
        - metadata.tsv.gz
        
Description of the folders is as follows:
 - [`raw`](#dataset_interaction_raw) : raw, plain-text software mentions, as extracted by the NER model
 - [`disambiguated`](#dataset_interaction_disambiguated): disambiguated software mentions, after disambiguation
 - [`linked`](#dataset_interaction_linked) :  linked software mentions
        
Note that for the folder `raw`, you don't have to have all of the `comm`, `non_comm`, and `publishers_collection` files. You can shoose as many as you would like to interact with. The description of these files is:

- `comm` : contains data extracted from the PMC OA **commercial** subset
- `non_comm` : contains data extracted from the PMC OA **non-commercial** subset
subset)
- `publishers_collection` : contains data extracted from the **CZI Publishers Collection**  

In the following sections, we will show how to interact with the different modalities of the dataset and touch on the motivation and different use cases each modality carries.

In [2]:
ROOT_DATA_DIR = '../data/'

<a id='dataset_interaction_raw'></a>
### raw files

The **raw** files contain raw, plain-text software mentions, as extracted by the NER model from each of the collections. As mentioned in the [Intro of this section](#dataset_interaction), there are three possible files to interact with:
 - comm_raw.tsv.gz
 - non_comm_raw.tsv.gz
 - publishers_collection_raw.tsv.gz
 
These files are quite large (~1GB zipped) and are stored as GZIP files. 
If you are using **pandas**, we recommend the following approach. <br>
Note that we offer an example of reading only the first 1000000 rows of the dataset, to speed up the computation. <br>
For full analysis of the dataset, you should take `nrows=num_rows_to_read` out and run:

In [3]:
#data_df = pd.read_csv(ROOT_DATA_DIR + 'disambiguated/comm_disambiguated.tsv.gz', sep = '\\t', engine = 'python', compression = 'gzip)

In [4]:
num_rows_to_read = 5000000
raw_df = pd.read_csv(ROOT_DATA_DIR + 'raw/comm_raw.tsv.gz', sep = '\\t', engine = 'python', compression = 'gzip', nrows = num_rows_to_read)

<color value = red>**Warning**</color>: It may take a while for the command to finish running (up to an hour), especially for the `comm` and `publishers_collection`. <br>
If you would like to quickly interact with the dataset, we recommend opening the `non_comm` file first, as it's the smallest one. <br><br>
Once the file is read, we can start exploring it! Let's look at a few samples:

In [5]:
raw_df.sample(3)

,license,location,pmcid,pmid,doi,pubdate,source,number,text,software,version,ID,curation_label
532885,comm,comm/Arch_Virol/PMC7588393.nxml,7588393,32951134.0,10.1007/s00705-020-04810-4,2020,NaN,2,"The database sequences were aligned using MAFFT version 7.453 [13] and used to compute a midpoint-rooted maximum-likelihood tree with 1,000 bootstrap replicates in IQ-TREE version 1.6.12 [14], using the ModelFinder [15] and ultrafast bootstrap [16] algorithms (Fig. 2)",MAFFT,NaN,SM1148,software
2334811,comm,comm/BMC_Public_Health/PMC3733870.nxml,3733870,23902931.0,10.1186/1471-2458-13-699,2013,Screening and baseline assessment,14,"Because AUDIT is reported to be less sensitive at identifying risk drinking in women [17], the cut-off points of binge drinking for women (4 units) were reduced by one unit as compared with men (5 units), as recommended by Freeborn and others [17]",AUDIT,NaN,SM3600,not_software
2866551,comm,comm/Biomed_Res_Int/PMC5742880.nxml,5742880,29379796.0,10.1155/2017/2978718,2017,fig_caption,0,[30]; Chimera software v1.6.2 was used for visualization,Chimera,NaN,SM1034,software


And the fields we have available:

In [6]:
raw_df.columns

Index(['license', 'location', 'pmcid', 'pmid', 'doi', 'pubdate', 'source',
       'number', 'text', 'software', 'version', 'ID', 'curation_label'],
      dtype='object')

Some field definitions that will be relevant for our work in this notebook: 
- **software** : the software mention extracted by the NER algorithm from **text**
- **version** : software version as extracted by the NER algorithm (note that not all mentions will have this)
- **text** : text that **software** is being extracted from
- **curation_label** : label assigned to software mention by our curators 

All of the other fields are related to metadata of the paper the mention is extracted from.  
For a full definition of all of the fields, please take a look at the [Dataset README.md](link) file in our [Github repo](https://github.com/chanzuckerberg/software-mentions). 

Now, let's look at a few examples of interacting with the dataset:
<a id='dataset_interaction_raw_query_scipy'></a>

#### 1.Query the dataset
Example of querying the dataset for the plain-text software mention 'scipy'

In [7]:
scipy_mentions_df = raw_df[raw_df['software'] == 'scipy']

In [8]:
scipy_mentions_df.head(3)

,license,location,pmcid,pmid,doi,pubdate,source,number,text,software,version,ID,curation_label
8488,comm,comm/ACS_Nano/PMC7905882.nxml,7905882,33556239.0,10.1021/acsnano.0c10632,2021,Cluster Analysis,40,The 83.4% confidence interval for the mean was calculated using the implementation in the scipy package,scipy,NaN,SM3076,software
9849,comm,comm/ACS_Synth_Biol/PMC8486170.nxml,8486170,33449631.0,10.1021/acssynbio.0c00318,2021,fig_caption,5,Wavelength was computed using scipy FFT and peak finding standard libraries,scipy,NaN,SM3076,software
9853,comm,comm/ACS_Synth_Biol/PMC8486170.nxml,8486170,33449631.0,10.1021/acssynbio.0c00318,2021,Computational Model,12,"Signal analysis (FFT) and visualization were carried out using the scipy and Matplotlib.pyplot libraries, respectively",scipy,NaN,SM3076,software


<a id='dataset_interaction_raw_samples_scipy'></a>
#### 2. Samples of texts in which the mention 'scipy'  appears

In [9]:
scipy_mentions_df[['pmid', 'source', 'text', 'software']].sample(10)

,pmid,source,text,software
3976467,31717800.0,2.9. Bioinformatics Analysis of Tissue-Selective Phages,"For each identified motif recovered per tissue, a 2 × 2 contingency table comparing the frequency of each motif observed in the isolated tissue with the frequency of each motif observed in the normalizing sample was prepared and a p-value calculated using a one-sided Fisher’s exact test, implemented in a custom script using the Fisher exact function from the scipy (v",scipy
1112654,29523077.0,Availability and requirements,"Other requirements: Python modules numpy 1.12.1, scipy 0.19.0, matplotlib 1.5.3, sklearn 0.18.1, networkx 1.11, community",scipy
3279291,34300770.0,4. Inverse Analysis,"Two sets of different types of optimization procedures have been implemented, both based on scipy.optimize module",scipy
275735,33042753.0,Statistical Analysis,Statistical analysis was performed using two‐tailed Student's t‐test (ttest_ind function of scipy package),scipy
699183,31660850.0,paper_abstract,Additional optimization methods from the Python scipy library are utilized as well,scipy
4396430,32200751.0,fig_caption,"To test for differences in nitrate, nitrite and ammonia/ammonium concentrations at time T24 the 10 communities in vertical and horizontal regimes (three independent measures per community) were compared using a two-sample Kolmogorov–Smirnov test (implemented in python 3.7.2, package scipy 1.3.1)",scipy
3809265,31830774.0,"Patients, phenotyping, and score validation","Custom Python scripts utilizing the itertools, scipy, and pandas libraries were used to perform the combinatorics analysis",scipy
3663615,32585128.0,RNA polymerase I profile,"Downstream analyses were performed using python 2.7 Jupiter notebooks, python libraries (pandas v0.19.2, numpy v1.16.0, scipy v1.2.0, matplotlib v2.2.3) and in-house scripts submitted as an update of gwide toolkit v0.5.27 (https://github.com/tturowski/gwide; Turowski et al., 2016)",scipy
1963273,31208429.0,Analysis,"The performance of each model was assessed using the Spearman correlation coefficient between the predicted and measured IC50 values in the testing set using the scipy statistics package version 0.17.0, p values were calculated within the statistics package using a student’s t distribution",scipy
1668652,32000670.0,K-mer analysis and chromosome painting,"Using scaled data for each of the 25 largest clusters, we calculated the Pearson’s and Spearman’s correlation coefficient between gene density and kmer mapping density using the scipy package in python 2.7.",scipy


<a id='dataset_interaction_raw_most_frequent_mentions'></a>
#### 3. Retrieve the most frequent software mentions in the corpus
Note that here, we define *frequency* by number of unique papers a software appears in

**Warning**: this command also takes a long time! Feel free to skip it if you're not interested in this section.

In [10]:
raw_df_aggregated = raw_df.groupby('software').nunique().sort_values(by = 'pmid', ascending = False)
raw_df_aggregated[:20]

,license,location,pmcid,pmid,doi,pubdate,source,number,text,version,ID,curation_label
software,,,,,,,,,,,,
SPSS,1,93333,93333,92987,92109,18,10332,357,97877,450,1,1
R,1,58276,58276,58161,58045,20,34297,348,94148,445,1,1
GraphPad Prism,1,31844,31844,31790,31692,18,8688,179,35670,218,1,1
Excel,1,28227,28227,28097,28010,19,12220,256,34465,153,1,1
SAS,1,25543,25543,25487,25359,19,5437,181,30612,143,1,1
ImageJ,1,22701,22701,22629,22560,18,19562,223,38242,345,1,1
BLAST,1,22648,22648,22603,22529,20,23158,205,44347,82,1,1
GraphPad,1,20055,20055,20031,19918,18,4830,158,20841,67,1,1
SHELXL97,1,19209,19209,19208,19209,15,65,50,7103,0,1,1


<a id='dataset_interaction_raw_curation_labels'></a>
#### 4. Understanding curation labels

Now, we may notice that in the most frequent software_mentions retrieved above, there are some obvious false positives picked up by the NER model, such as **'COVID'**, **'COVID-19'** or **'Google'**. This is why we engaged an expert team of biomedical curators to sanity-check the top 10k plain-text software mentions extracted by the NER model from the `comm` dataset. We offer more details about the methodology and guidelines in our [preprint](link), as well as on our [Github Repo](https://github.com/chanzuckerberg/software-mentions). The datasets contain a field ```curation_label``` that tells us the label our curation team assigned to a software mention, or if it was not curated at all. 

Let's explore these **curation_label** a bit. First, let's see what values are available

In [11]:
raw_df['curation_label'].unique()

array(['not_curated', 'unclear', 'software', 'not_software'], dtype=object)

Now, let's create the **mention2curation_label** mapping to map from a mention to its curation label

In [12]:
mentions_curation_label_df = raw_df[['software', 'curation_label']].drop_duplicates()
mentions = mentions_curation_label_df['software'].values
curation_labels = mentions_curation_label_df['curation_label'].values
mention2curation_label = {m : c for m, c in zip(mentions, curation_labels)}

Let's look at curation labels of the top most frequent mentions obtained above:

In [13]:
raw_df_aggregated = raw_df_aggregated.reset_index()
raw_df_aggregated['curation_label'] = raw_df_aggregated['software'].apply(lambda x: mention2curation_label[x])
raw_df_aggregated[:20]

,software,license,location,pmcid,pmid,doi,pubdate,source,number,text,version,ID,curation_label
0,SPSS,1,93333,93333,92987,92109,18,10332,357,97877,450,1,software
1,R,1,58276,58276,58161,58045,20,34297,348,94148,445,1,software
2,GraphPad Prism,1,31844,31844,31790,31692,18,8688,179,35670,218,1,software
3,Excel,1,28227,28227,28097,28010,19,12220,256,34465,153,1,software
4,SAS,1,25543,25543,25487,25359,19,5437,181,30612,143,1,unclear
5,ImageJ,1,22701,22701,22629,22560,18,19562,223,38242,345,1,software
6,BLAST,1,22648,22648,22603,22529,20,23158,205,44347,82,1,software
7,GraphPad,1,20055,20055,20031,19918,18,4830,158,20841,67,1,software
8,SHELXL97,1,19209,19209,19208,19209,15,65,50,7103,0,1,software
9,Stata,1,17766,17766,17733,17734,19,3205,156,20449,219,1,software


We can see that the mentions `COVID`, `COVID-19`, `Google Scholar`, as well as `Google` have been correctly marked by our curators as **not_software**. <br>
We can now segment the dataset based on the curation_labels:

In [14]:
not_software_raw = raw_df[raw_df['curation_label'] == 'not_software']
unclear_raw = raw_df[raw_df['curation_label'] == 'unclear']
not_curated_raw = raw_df[raw_df['curation_label'] == 'not_curated']
software_raw = raw_df[raw_df['curation_label'] == 'software']

Let's look at some more examples of mentions that have been marked by our curators as **not_software** and the context in which they appear:

In [15]:
not_software_raw.sample(10)[['pmid', 'text', 'software']]

,pmid,text,software
3765342,30813336.0,"The objective of OPLS-DA is to separate the systematic variation in X into two parts, one part which is linearly related to Y, and another part is orthogonal to Y, which leads to better class resolution in a discriminant problem [31]",OPLS-DA
3209673,32019541.0,"Genetic polymorphisms and the natural selection of PvLDH and PfLDH were analysed using DNASTAR, MEGA6, and DnaSP ver",DNASTAR
1868966,21961925.0,Key organizational factors at local VHA medical centers were strongly associated with MOVE! implementation,MOVE!
4806164,23028359.0,Genes repressed by COMPASS therefore show abberant H3K4 methylation patterns that are characterized by a reversed orientation of the normal H3K4 methylation pattern observed for active genes [3].,COMPASS
1904253,28284209.0,"STARD [17] diagram DENV-4 patients in Rio de Janeiro, 2013",STARD
2036046,31533828.0,"This toolkit consisted of a PtDA (MyDiabetesPlan) and its accompanying implementation tools (such as how-to videos, and enabler cards with step-by-step instructions)",PtDA
4926277,27529696.0,"Data were analyzed using FlowJo (Treestar), and counts of each population (Tv, T-cell or B-cell) were determined according to the gating strategy is shown in S1 Fig",Treestar
4904976,32614857.0,"The models were parameterized with age-structured data on approximately 21,000 individuals included in the KalaNet bednet trial in India and Nepal [37] and have undergone geographical cross-validation against data on >5000 VL cases from 8 endemic districts in Bihar collected by CARE India [38] (see [36] for full model code and descriptions, and sensitivity analyses)",CARE
4400637,33100806.0,The contagion airborne transmission (CAT) inequality that evaluates the conditions for the airborne transmission of a respiratory infection such as COVID-19,COVID
1935657,24886446.0,"The entire compilation of CanMEDS 2005 text documents were accessed on the website of the Royal College of Physicians and Surgeons of Canada [21] on July 31, 2013",CanMEDS


Similarly, we can look at examples of mentions that have been marked by our curators as **software**:

In [16]:
software_raw.sample(10)[['pmid', 'text', 'software']]

,pmid,text,software
315987,34176789.0,"Furthermore, six core gene targets including CAT, NOS2, CXCR3, MAPK1, GPT, and ICAM1 of VA against CHOL/COVID-19 were identified using Cytoscape tool (Figure 4B).",Cytoscape
2570083,24847907.0,Immunoreactive bands were visualized using enhanced chemiluminescence (ECL; Amersham Pharmacia Biotech) and quantified by NIH image software,NIH image
4969874,22479336.0,"(3) Number of males released, because our VORTEX model results suggested, perhaps counter-intuitively, that number of adult males released should have no effect on reintroduction success.",VORTEX
2569649,30034650.0,Graph Pad Prism software version 4.03 was used for statistical analysis,Graph Pad Prism
3814021,29948643.0,PET image reconstruction were performed using an OSEM 3D/OP-MAP with scatter correction and a ramp filter (matrix size 128 × 128),OSEM
1440449,17459166.0,Log2 measures of gene expression were normalized using a linear mixed model in SAS (JMP v6.0.0 with a microarray platform beta-version in SAS v9.1.3) to remove the effects of dye (fixed effect) and array (random effect) following a joint regional and spatial Lowess transformation in MAANOVA Version 0.98.8 for R to account for both intensity and spatial bias [43],MAANOVA
3849646,32281725.0,Alignment was performed with ClustalW based on identical residues,ClustalW
4976099,25243403.0,"Therefore, ELASPIC should not be taken as a predictor of disease mutations, such as the sequence-based methods MutationTaster [56] or SNAP [57], since it was trained to predict stability rather than phenotype effect of mutations",MutationTaster
1374396,23642277.0,Interviews were transcribed verbatim anonymously and analyzed using QSR International's NVivo 9 software [13],NVivo
4448333,29989238.0,"Chains of syntenic genes were identified using DAGchainer (Haas et al., 2004), allowing a maximum gene distance of 20 between two matches and with a minimum number of four aligned pairs constituting a syntenic block",DAGchainer


Same for **unclear**:

In [17]:
unclear_raw.sample(10)[['pmid', 'text', 'software']]

,pmid,text,software
4698095,33370252.0,"Moco contains additional cost modules beyond what is shown here, and users can define their own custom modules.",Moco
433488,33430096.0,"Analyses were performed on a BD FACSCanto™ II (BD Biosciences, Heidelberg, Germany) equipped with a 488 nm blue laser, 630 nm red laser, and a 405 nm violet laser, and by using BD FACSDiva™ Software II (BD Biosciences)",BD
4599589,32287273.0,a) DOT assigns lower p-values than other methods do to the 4 selected breast cancer genes,DOT
3216322,18062817.0,Box plots of the anomalies of the DEMETER re-forecasts over Kagera region,DEMETER
2189796,25204857.0,"Studies up to December 31, 2012 were retrieved using the following 11 electronic databases: (1) Medline, (2) CINAHL, (3) Scopus, (4) Academic Search Complete, (5) Educational Research Complete, (6) Web of Science, (7) Sport Discus, (8) ERIC, (9) LILACS, (10) Cochrane Central Register of Controlled Trials (CENTRAL) and (11) Proquest",Web
4016972,33693062.0,Is the Open Letter written in accessible language?,Open
4343195,33816840.0,"The results of our empirical study show that only very few LFR methods actually contain a fault, and thus, they indicate that IDP can successfully identify methods that are not fault-prone",IDP
2136037,19761597.0,All analyses were performed with SAS version 9.1,SAS
2586281,28342072.0,Example SAS code for all Monte Carlo studies in this manuscript is available online at https://figshare.com/s/8d48fed4a23fff78e2a3 .,SAS
3645905,23396999.0,Analysis of variance (ANOVA) was performed in SAS v.9.2 (SAS Institute 2010) to develop least square means for genotypes grown in 2011 near Toluca (experimental design Randomized Complete Block Design; genotype as fixed and replication as random variables),SAS


Note that for mentions that are marked as **unclear**, we don't recommend excluding them from analyses. They should rather be interpreted as *it cannot be assumed that this plain-text software mention will always be a true software mention when appearing in text*. The curators have only been provided with 5 sentences per software mention, and they did not curate each individual sentence in which a mention appears. The evaluations are based solely on those 5 sentences. We offer a more in-depth discussion about this in our [preprint](link) and [curation documents](link)

This means that if we wanted to clean our dataset, we should only exclude the mentions markes as **not_software**:

In [18]:
raw_df_clean = raw_df[raw_df['curation_label'] != 'not_software']

Looking at the top mentions again:

In [19]:
raw_df_clean_aggregated = raw_df_clean.groupby('software').nunique().sort_values(by = 'pmid', ascending = False)
raw_df_clean_aggregated[:20]

,license,location,pmcid,pmid,doi,pubdate,source,number,text,version,ID,curation_label
software,,,,,,,,,,,,
SPSS,1,93333,93333,92987,92109,18,10332,357,97877,450,1,1
R,1,58276,58276,58161,58045,20,34297,348,94148,445,1,1
GraphPad Prism,1,31844,31844,31790,31692,18,8688,179,35670,218,1,1
Excel,1,28227,28227,28097,28010,19,12220,256,34465,153,1,1
SAS,1,25543,25543,25487,25359,19,5437,181,30612,143,1,1
ImageJ,1,22701,22701,22629,22560,18,19562,223,38242,345,1,1
BLAST,1,22648,22648,22603,22529,20,23158,205,44347,82,1,1
GraphPad,1,20055,20055,20031,19918,18,4830,158,20841,67,1,1
SHELXL97,1,19209,19209,19208,19209,15,65,50,7103,0,1,1


<a id='dataset_interaction_disambiguated_freq_samples'></a>
Now, by looking at the most frequent mentions on the dataset above, we can make another observation. There seem to be plain-text software mentions that point to the same software. For Instance:
- **SPSS** and **SPSS Statistics** are the same software **SPSS**, 
- **GraphPad Prism**, **GraphPad** and **Prism** are likely pointing to the same software **GraphPad Prism**
- **MATLAB** and **Matlab** 
- **ImageJ** and **Image J**

If we want to gauge the full impact of a piece of software, we should aggregate the impact of all of its string variations. This is the issue we address in the section below.

<a id='dataset_interaction_disambiguated'></a>
### disambiguated files

Through disambiguation, the goal is to cluster together plain-text software mentions that point to the same software entity. In this section, we go over a [short description of the motivation](#dataset_interaction_disambiguated_motivation'), as well as some examples of engaging with the disambiguated dataset. More in-depth details about the disambiguation algorithm, which is based on string similarity algorithms and [DBSCAN](#references_dbscan), can be found in our preprint, as well as under our Github repository page.


The **disambiguated** files contain plain-text mentions, as extracted by the NER model, mapped to software entities, as determined by our disambiguation algorithm. There is only one file to interact with:
- comm_disambiguated.tsv.gz

The file is quite large and stored as GZIP archive. Similarly to the [**raw** datasets](#dataset_interaction_raw), if you are using pandas, we recommend opening using the following. We also offer an example of reading only the first 1000000 rows. For querying the entire dataset, please remove this constraint and run:

In [20]:
#disambiguated_df = pd.read_csv(ROOT_DATA_DIR + 'disambiguated/comm_disambiguated.tsv.gz', sep = '\\t', engine = 'python', compression = 'gzip')

In [ ]:
disambiguated_df = pd.read_csv(ROOT_DATA_DIR + 'disambiguated/comm_disambiguated.tsv.gz', sep = '\\t', engine = 'python', compression = 'gzip', nrows = 10000000)

Note that due to its size, reading the entire file will take quite a long time to open (~20 min)

Let's start by looking at a few samples:

In [ ]:
pd.set_option('max_colwidth', 50)
disambiguated_df.sample(3)

We can look at the fields we have available

In [ ]:
disambiguated_df.columns

The fields in this dataset are largely the same as the ones in the **raw** dataset. As a reminder: 
- **software** : the software mention extracted by the NER algorithm from **text**
- **version** : software version as extracted by the NER algorithm (note that not all mentions will have this)
- **text** : text that **software** is being extracted from
- **curation_label** : label assigned to software mention by our curators 

These are all fields we've seen before in the **raw** dataset. However, now we get an extra field:

- **mapped_to_software** : software entity (or cluster) the **software** mention is predicted to be part of; this is the result of disambiguation

As a reminder, for a full definition of all of the fields, please take a look at the [Dataset_README.md](link) file in our [Github repo](https://github.com/chanzuckerberg/software-mentions). 

Now let's get started exploring the disambiguated dataset!

For mentions we were not able to disambiguate, we set the **mapped_to_software** field to the value of the **software** mention itself.

In [ ]:
def populate_map_to_software(x):
    if x['mapped_to_software'] == 'not_disambiguated':
        return x['software']
    return x['mapped_to_software']

In [ ]:
disambiguated_df['mapped_to_software'] = disambiguated_df.apply(populate_map_to_software, axis = 1)

<a id='dataset_interaction_disambiguated_motivation'></a>
#### 1. Why do we need disambiguation? 

First, let's understand more why we would want to disambiguate the software mentions. 
[In the previous section](#dataset_interaction_disambiguated_freq_samples), we already started to see that the same software can be extracted by the NER model from a paper under different string variations. Let's look at a few more examples, to drive the point home.

<a id='dataset_interaction_disambiguated_motivation_scipy'></a>
##### SciPy
We have already looked at **SciPy** in the previous section. But *'SciPy'* is not the only string variation authors can mention the software **SciPy** in their papers, or in which the NER model might extract a string that points to the **SciPy** software entry. Some possible variations that the NER model could extract are: `['scipy', 'Scipy', 'SciPy', 'SCIPY']`. Let's see  how these can appear in the dataset. <br> 
Note that for simplification, we're only outputting the **pmid**, **text** and **software**.

In [ ]:
relevant_fields = ['pmid', 'text', 'software']
num_samples = 5
raw_df[raw_df['software'] == 'scipy'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'Scipy'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'SciPy'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'SCIPY'][relevant_fields][:num_samples]

Let's look at a few more examples! Note that in each case, we are only showcasing a **few** string variations as an example. However, software entities can have hundreds of string variations, as extracted by the NER model. For more examples of this and a discussion, please visit our preprint.

<a id='dataset_interaction_disambiguated_motivation_BLAST'></a>
##### BLAST
Some example of string variations can be: `['BLAST', '(BLAST)', 'BLAST Whole Genome', 'BLAST) search']`

In [ ]:
raw_df[raw_df['software'] == 'BLAST'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == '(BLAST)'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'BLAST Whole Genome'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'BLAST) search'][relevant_fields][:num_samples]

<a id='dataset_interaction_disambiguated_motivation_sklearn'></a>
##### scikit-learn

Example string variations:
`['scikit-learn', 'scikit-learn python package', 'scikit-learn python library', 'scikit-learn python', 'scikit-learn library for Python', 'scikit-learn Python package2223']`

In [ ]:
raw_df[raw_df['software'] == 'scikit-learn'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'scikit-learn python package'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'scikit-learn python library'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'scikit-learn python'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'scikit-learn library for Python'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'scikit-learn Python package2223'][relevant_fields][:num_samples]

<a id='dataset_interaction_disambiguated_motivation_ImageJ'></a>
##### ImageJ

Example string variations:
`['ImageJ', 'Image J']`

In [ ]:
raw_df[raw_df['software'] == 'ImageJ'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'Image J'][relevant_fields][:num_samples]

<a id='dataset_interaction_disambiguated_motivation_MATLAB'></a>
#### MATLAB
Example string variations:
`['MaTLab', 'MatLAB', 'MatLab', 'Matlab', 'MATLAB, Statistics and Machine Learning Toolbox', 'mAtLab)', 'Matlab)']`

In [ ]:
raw_df[raw_df['software'] == 'MaTLab'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'MatLAB'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'MaTLab'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'MatLab'][relevant_fields][:num_samples]

In [ ]:
raw_df[raw_df['software'] == 'Matlab'][relevant_fields][:num_samples]

<a id='dataset_interaction_disambiguated_dataset_query'></a>
#### Query the dataset for a particular software, including all string variations

Great! So now that we saw the benefit of disambiguating the software terms, let's see how we can query our disambiguated dataset and interact with it. Let's start by going back to searching for the **scipy** entry. <BR>

In [ ]:
software = 'scipy'
disambiguated_df[disambiguated_df['software'] == software].head(3)

We see that the **mapped_to_software** field for the entries in which **software** is **scipy** is **SciPy**, which means that the plain-text software mention **scipy** got mapped to the **SciPy** software entity through disambiguation. We can also obtain this by:

In [ ]:
mapped_to = disambiguated_df[disambiguated_df['software'] == software]['mapped_to_software'].values[0]
print('The', software, 'mention got mapped to the', mapped_to, 'software entity')

In [ ]:
mapped_to

Now, let's look at all the other entries that got mapped to the same entry:

In [ ]:
mapped_to_df = disambiguated_df[disambiguated_df['mapped_to_software'] == mapped_to]

Now we can retrieve all of the string variations that map to the same entry:

In [ ]:
string_variations = mapped_to_df['software'].unique()
print('Here are other potential string variations for', software, 'as given by the disambiguation algorithm')
print(string_variations)

<a id='dataset_interaction_disambiguated_examples'></a>
#### Examples of disambiguated software terms

Now let's look at a few more examples! For the software mentions below, we will retrieve the software entity they get mapped to, and what other string variations they can appear under

In [ ]:
software_mentions = ['BLAST', 'sklearn', 'SciPy', 'SPSS', 'ImageJ', 'limma', 'BERT', 'scikit-image', 
                    'GraphPad Prism']

for software in software_mentions:
    mapped_to = disambiguated_df[disambiguated_df['software'] == software]['mapped_to_software'].values[0]
    mapped_to_df = disambiguated_df[disambiguated_df['mapped_to_software'] == mapped_to]
    string_variations = mapped_to_df['software'].unique()
    print(software, 'got mapped to the', mapped_to, 'software entity')
    print('There are', len(string_variations), 'other potential string variations')
    print('Some of these are:')
    print('='*30)
    print(string_variations[:50]) #only printing 50 for visibility

<a id='dataset_interaction_disambiguated_most_frequent_mentions'></a>

#### Example: Retrieve the most frequent software mentions based on disambiguated dataset

Now let's look at getting the most frequent terms in our dataset, bassed on the **mapped_to** field
It is the same command as on the [**raw** dataset](#dataset_interaction_raw_most_frequent_mentions), except that now we change the **software** field for the **mapped_to_software** field. Now the counts should be aggregated over *all* the string variations, as outputted by the disambiguation algorithm.

In [ ]:
disambiguated_df_aggregated = raw_df.groupby('software').nunique().sort_values(by = 'pmid', ascending = False)
disambiguated_df_aggregated[:20]

Let's remind ourselves the most frequent terms on the raw dataset:

In [ ]:
raw_df_aggregated[:20]

Note that the counts on the **disambiguated** and the **raw** dataset splits are different. This is because now, we are aggregating over multiple string variations for each string. 

<a id='dataset_interaction_linked'></a>
### linked files

We queried a number of databases, searching for exact matches for plain text sofware mentions in the `comm` collection. The databases we queryied are: 
- [PyPI Index](https://pypi.org/simple/)
- [Bioconductor Index](https://www.bioconductor.org/packages/release/bioc/)
- [CRAN Index](https://cran.r-project.org/web/packages/available_packages_by_name.html)
- [GitHub API](https://github.com)
- [SciCrunch API](https://scicrunch.org/resources)

We describe in detail the methodology we used to obtain the links in our preprint, as well as in our [Github Repository](https://github.com/chanzuckerberg/software-mentions/tree/main/software-mentions-linker-disambiguator)

We compiled this information in the `metadata.tsv.gz` file, which is assumed to be under the `linked` directory, as described in the [Interacting with the dataset Section](#dataset_interaction)

The **linked** files contain metadata links, extracted through the linking algorithm. There is only one file to interact with:
- metadata.tsv.gz

The file is also stored as GZIP archive. Similarly to the [**raw** datasets](#dataset_interaction_raw), if you are using pandas, we recommend opening using the following:

In [ ]:
linked_df = pd.read_csv(ROOT_DATA_DIR + 'linked/metadata.tsv.gz', sep = '\\t', engine = 'python', compression = 'gzip')

Let's start by looking at a few samples:

In [ ]:
linked_df.sample(3)

We can look at the fields we have available

In [ ]:
linked_df.columns

We describe the fields in details on the Github repository, under [Linking Schema](https://github.com/chanzuckerberg/software-mentions/tree/main/software-mentions-linker-disambiguator#linking-schema)

Now let's explore the information we have available!

<a id='dataset_interaction_linked'></a>
#### Examples of linked software terms

Let's start by querying a number of software mentions, to see what links we get. We can start with the previous set of software mentions we looked at: ***'BLAST', 'sklearn', 'SciPy', 'SPSS', 'ImageJ', 'limma', 'BERT', 'scikit-image', 'GraphPad Prism'***

##### BLAST

In [ ]:
linked_df[linked_df['software'] == 'BLAST']

##### sklearn

In [ ]:
linked_df[linked_df['software'] == 'sklearn']

##### SciPy

In [ ]:
linked_df[linked_df['software'] == 'SciPy']

##### SPSS

In [ ]:
linked_df[linked_df['software'] == 'SPSS']

##### ImageJ

In [ ]:
linked_df[linked_df['software'] == 'ImageJ']

##### limma

In [ ]:
linked_df[linked_df['software'] == 'limma']

##### BERT

In [ ]:
linked_df[linked_df['software'] == 'BERT']

##### scikit-image

In [ ]:
linked_df[linked_df['software'] == 'scikit-image']

<a id='dataset_interaction_linked_metadata_fields'></a>
#### Exploration of metadata fields

In this section, we want to offer some examples of the type of metadata we are able to retrieve through the links we created. <br>
First, let's remind ourselves the fields we have available:

In [ ]:
linked_df.columns

And here is a full description of the metadata fields:

| field | description |
| :--- | :--- |
| **ID** 	| unique ID of software mention (generated by us) |
| **software_mention**	| plain-text software mention |
| **mapped_to**	| value the software_mention is being mapped to |
| **source**	| source of the mapping - eg Bioconductor Index, GitHub API |
| **platform**	| platform of software_mention - eg PyPI, CRAN |
| **package_url**	| URL linking software_mention to source |
| **description**	| description of software_mention |
| **homepage_url**	| homepage_url of software_mention |
| **other_urls**	| other related URLs |
| **license**	| software license |
| **github_repo**	| GitHub repository |
| **github_repo_license**	| GitHub repository license |
| **exact_match**	| whether or not this mapping was an exact match |
| **RRID**	| RRID for software_mention |
| **reference**	| journal articles linked to software_mention (identified either through DOI, pmid or RRID) |
| **scicrunch_synonyms**	| synonyms for software_mention, retrieved from Scicrunch |

##### package_url + description
Examples for the software entities:

In [ ]:
linked_df[linked_df['software'] == software][['software', 'package_url', 'description']]

##### homepage_url + other_urls
Examples for the software entities:

In [ ]:
linked_df[linked_df['software'] == software][['software', 'homepage_url', 'other_urls']]

##### github_repo + github_repo_license
Examples for the software entities:

In [ ]:
linked_df[linked_df['software'] == software][['software', 'github_repo', 'github_repo_license']]

##### RRID
Examples for the software entities:

In [ ]:
linked_df[linked_df['software'] == software][['software', 'RRID']]

##### reference
Examples for the software entities:

In [ ]:
linked_df[linked_df['software'] == software][['software', 'reference']]

##### scicrunch_synonyms
Examples for the software entities:

In [ ]:
linked_df[linked_df['software'] == software][['software', 'scicrunch_synonyms']]

##### platform
Examples for the software entities:

In [ ]:
linked_df[linked_df['software'] == software][['software', 'platform']]

##### license
Examples for the software entities:

In [ ]:
linked_df[linked_df['software'] == software][['software', 'license']]

<a id='dataset_interaction_linked_query'></a>
## Query the dataset for a particular software, retrieving all the data
In this section, we offer an example of querying the dataset end2end for a particular software entitiy, retrieving all of the available information <br>
Let's take for example the software_mention **scikit-learn**


###### 1. Retrieve software entity mapping

In [ ]:
software_mention = 'scikit-learn'
mention_df = disambiguated_df[disambiguated_df['software'] == software_mention]

In [ ]:
software_mention_mapping = disambiguated_df[disambiguated_df['software'] == software_mention]['mapped_to'].values[0]
print('Mapped', software_mention, 'to', software_mention_mapping)

###### 2. Retrieve string variations

In [ ]:
string_variations = disambiguated_df[disambiguated_df['software'] == software_mention_mapping]['software'].unique
print('String variations for', software_mention_mapping)
print('=' * 30)
print(string_variations)

###### 3. Retrieve total number of unique publications mention appears in

###### undisambiguated

In [ ]:
num_counts_undisambiguated = disambiguated_df[disambiguated_df['software'] == software_mention]['pmid'].nunique()
print(software_mention, 'appears in', num_counts_undisambiguated, 'number of PMIDs')

###### disambiguated

In [ ]:
num_counts_disambiguated = disambiguated_df[disambiguated_df['software'] == software_mention_mapping]['pmid'].nunique()
print(software_mention, 'is mapped to', software_mention_mapping, 'which appears in', num_counts_disambiguated, 'number of PMIDs')

###### 4. Retrieve metadata

In [ ]:
software_mention_medatada = linked_df[linked_df['software'] == software_mention]

In [ ]:
software_mention_medatada

<a id='prompts'></a>

That's it! This notebook serves as a starting point for showcasing how particular research questions might be answered using the data available in the **CZI Software Mentions Dataset**. It is our hope that it sparks further analyses and interesting research directions. <br>

**A non-exhaustive list potential questions to consider next is:**
1. Understanding the context in which a software mention appears. Ideas include using topic modeling, pretrained language models embeddings or models for citation intent classification. 
2. Drawing better insights about usage of software in particular fields by incorporating other metadata about a paper, such as  MESH terms, author provided-keywords, title, or abstract
3. Explore measure of impact for software, such as Eigenfactor
4. Looking into open-access policies of most used or impactful software, and how that varies in particular fields
5. Exploring differences in how software usage varies over time


We are excited about the work that will build on top of the **CZI Software Mentions Dataset** and can't wait to see what other ideas you come up with!

### References

<a id='references_scibert'>1. Beltagy, Iz, Kyle Lo, and Arman Cohan. "SciBERT: A pretrained language model for scientific text." arXiv preprint arXiv:1903.10676 (2019).</a> <br>
<a id='references_dbscan'>2. Ester, Martin, et al. "A density-based algorithm for discovering clusters in large spatial databases with noise." kdd. Vol. 96. No. 34. 1996.</a>